In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .master("local[*]")
        .appName("UDF")
        .config("spark.log.level", "WARN")
        .getOrCreate()
)

25/03/13 16:36:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
train = spark.read.csv("data/train.csv", inferSchema=True, header=True).cache()

In [3]:
train.show(5)

+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|timestamp|user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|     0|        0|    115|      5692|              0|                1|          3|                 1|                       NULL|                          NULL|
|     1|    56943|    115|      5716|              0|                2|          2|                 1|                    37000.0|                         false|
|     2|   118363|    115|       128|              0|                0|          0|                 1|                    55000.0|                         false|
|     3|   131167|    115|  

In [4]:
from pyspark.sql import functions as f
from pyspark.sql.functions import col
from pyspark.sql.types import LongType

def to_months(ms):
    return ms // 31536000000 * 12

to_months_udf = f.udf(to_months, LongType())

In [5]:
%%time
(
    train
        .select("content_id", to_months_udf("timestamp").alias("month"))
        .groupBy("content_id")
        .mean("month")
        .show(10)
)

+----------+------------------+
|content_id|        avg(month)|
+----------+------------------+
|      4519|0.4460020768431983|
|      4818|0.5068813295247988|
|      5518|0.6536082474226804|
|     13285| 2.725925925925926|
|     12027|2.2696177062374248|
|       833|0.7334516865330967|
|      9427| 0.652203699474766|
|       496|0.6270834569072898|
|      5156|0.5233297393761386|
|      6336|0.6127002288329519|
|      2866|0.9061162849232318|
|      1959|1.0353606789250354|
|      7982|0.6966064981949458|
|     23336| 0.564428149870339|
|       148| 0.602506180206291|
|      1342|0.7789912124752375|
|      1088|0.7588338484671289|
|      1580|0.7253544345532477|
|       471|0.8491909385113269|
|      2122|0.8964813870474247|
+----------+------------------+
only showing top 20 rows

CPU times: user 34.1 ms, sys: 21.7 ms, total: 55.8 ms
Wall time: 54.7 s


In [6]:
%%time
(
 train
    .select("content_id", (col("timestamp") / 31536000000 * 12).alias("month"))
    .groupBy("content_id")
    .mean("month")
    .show(10)
)

+----------+------------------+
|content_id|        avg(month)|
+----------+------------------+
|      4519|2.2218736221822732|
|      4818|2.3372134952525556|
|      5518| 3.029691836679273|
|     13285| 5.964229418381344|
|     12027|5.9773868010962765|
|       833|  3.25216382900438|
|      9427| 3.108803192689145|
|       496|2.9023706505861853|
|      5156|2.6012427887203198|
|      6336|2.7640873674707946|
+----------+------------------+
only showing top 10 rows

CPU times: user 11.7 ms, sys: 5.9 ms, total: 17.6 ms
Wall time: 4.07 s
